In [ ]:
import numpy as np


class VulnerableNeuralCrypto:
    def __init__(self, key_bits):
        """
        key_bits: 공격자가 찾아내야 할 비밀 키 (0 또는 1로 구성된 리스트)
        """
        self.key = np.array(key_bits, dtype=float)


    def relu(self, x):
        return np.maximum(0, x)
    
    def forward(self, input_vector):
        """
        보고서의 3.1.1 섹션에 기반한 XOR 구현 (Natural Implementation)
        y = ReLU(x - k) + ReLU(k - x)
        """
        # 입력 벡터와 키 벡터간의 Element-wise 연산
        term1 = self.relu(input_vector - self.key)
        term2 = self.relu(self.key - input_vector)

        # 이것이 신경망이 계산하는 암호화된 출력(Ciphertext)의 일부입니다.
        return term1 + term2
    
    # 시뮬레이션 설정: 8비트 비밀키 설정 (예: 10110010)
secret_key = [1, 0, 1, 1, 0, 0, 1, 0]
black_box = VulnerableNeuralCrypto(secret_key)
print(f"Target Secret key: {secret_key}")

Target Secret key: [1, 0, 1, 1, 0, 0, 1, 0]


In [3]:
def jitter_attack(model, input_dim, epsilon=0.001):
    """
    보고서 4.2 'The Jitter Attack' 구현
    
    model: 공격 대상 (Black Box)
    input_dim : 입력 벡터의 차원 (비트 수)
    epslion : 입력에 주입할 미세한 노이즈 (Jitter)
    """
    recovered_key = []

    # 공격자는 모든 입력을 0으로 설정한 기준 벡터를 생성합니다.
    base_input = np.zeros(input_dim)

    print(f"\n--- Starting Jitter Attack (epsilon={epsilon}) ---")
    for i in range(input_dim):
        # 1. 기준점(x=0)에서의 출력
        # 공격자는 i번째 비트에만 관심이 있으므로 해당 위치를 조작합니다.

        # f(0)
        input_base = base_input.copy()
        output_base = model.forward(input_base)[i]

        # 2. 양의 방향 지터 주입: f(0 + epsilon)
        input_pos = base_input.copy()
        input_pos[i] += epsilon
        output_pos = model.forward(input_pos)[i]

        # 3. 음의 방향 지터 주입: f(0 - epsilon)
        input_neg = base_input.copy()
        input_neg[i] -= epsilon
        output_neg = model.forward(input_neg)[i]

        # 4. 비선형성(Curvature) 측정
        # Key가 0이면 V Shape의 바닥이므로 노이즈에 민감하게 반응(비선형적 변화)
        # Key가 1이면 선형 구간이므로 노이즈가 선형적으로 상쇄됨
        curvature = output_pos + output_neg - (2 * output_base)

        # 분석 (보고서 4.2.1 시나리오 A vs B)
        # 이론적으로 Key=0일 때, curvature는 2*epslion, Key=1일 때 0에 가까움
        if abs(curvature) > epsilon: # 임계값은 epsilon 정도로 설정
            predicted_bit = 0
            status = "V-Shaped Detected (Non-linear)"

        else:
            predicted_bit = 1
            status = "Linear Slope Detected"

        print(f"Bit {i}: Curvature={curvature:.5f} -> Predicted Key: {predicted_bit} [{status}]")
        recovered_key.append(int(predicted_bit))
    return recovered_key

# 공격 실행
recovered_key = jitter_attack(black_box, input_dim=len(secret_key))
print("\n--- Verification ---")
print(f"\nOriginal Key : {secret_key}")
print(f"Recovered Key: {recovered_key}")


if secret_key != recovered_key:
    print("\n[ERROR] Keys do not match!")
    for i in range(len(secret_key)):
        if secret_key[i] != recovered_key[i]:
            print(f"Mismatch at index {i}: Secret={secret_key[i]}, Recovered={recovered_key[i]}")
else:
    print("\n[SUCCESS] Assertion Passed: Keys are identical.")

# 검증
assert secret_key == recovered_key
print("\n[SUCCESS] All key bits recovered via Jitter Attack!")


--- Starting Jitter Attack (epsilon=0.001) ---
Bit 0: Curvature=0.00000 -> Predicted Key: 1 [Linear Slope Detected]
Bit 1: Curvature=0.00200 -> Predicted Key: 0 [V-Shaped Detected (Non-linear)]
Bit 2: Curvature=0.00000 -> Predicted Key: 1 [Linear Slope Detected]
Bit 3: Curvature=0.00000 -> Predicted Key: 1 [Linear Slope Detected]
Bit 4: Curvature=0.00200 -> Predicted Key: 0 [V-Shaped Detected (Non-linear)]
Bit 5: Curvature=0.00200 -> Predicted Key: 0 [V-Shaped Detected (Non-linear)]
Bit 6: Curvature=0.00000 -> Predicted Key: 1 [Linear Slope Detected]
Bit 7: Curvature=0.00200 -> Predicted Key: 0 [V-Shaped Detected (Non-linear)]

--- Verification ---

Original Key : [1, 0, 1, 1, 0, 0, 1, 0]
Recovered Key: [1, 0, 1, 1, 0, 0, 1, 0]

[SUCCESS] Assertion Passed: Keys are identical.

[SUCCESS] All key bits recovered via Jitter Attack!
